# Frequency Asked Questions

## How do I import SynopticPy?
Simply `import synoptic` to get access to all the services classes. The **datetime** library is very useful. Oh, and you will very likely need to import **polars** for DataFrame manipulation.

In [29]:
from datetime import datetime, timedelta

import synoptic
import polars as pl

## What Synoptic weather API services are available?

All the Synoptic Weather API services are available with the exception of `qcsegments`, which is not implemented (I have never needed it). There are essentially two types of API services.

Data Services

- `synoptic.TimeSeries` : Request time series data from a station or stations.
- `synoptic.Latest` :  Request latest data from a station or stations.
- `synoptic.NearestTime` :  Request data from a station or stations nearest a specific time. Very similar to the Latest service.
- `synoptic.Precipitation` : Request precipitation data from a station or stations.
- `synoptic.Latency` : Request latency information for a station or stations.
- `synoptic.Metadata` : Request station metadata, like location, name, etc.

Metadata Services

- `synoptic.QCTypes` : Table of Synoptic's quality control types.
- `synoptic.Variables` : Table of Synoptic's variable definitions.
- `synoptic.Networks` : Table of Synoptic's available networks.
- `synoptic.NetworkTypes` : Table of Synoptic's network type categories.


## What is included in a Synoptic Services class instance?

In general, most instances return the following attributes:

1. All capitalized attributes like `SUMMARY`,  `STATION`, `UNITS`, `QC_SUMMARY` are copied dictionaries attached from the returned json. These are for convenience.
1. `df` is the long-format Polars DataFrame of the `STATION` data.
1. `endpoint` is the URL for the requested API service.
1. `help_url` is the website for the documentation for the service.
1. `json` is the returned json from the API request loaded into a Python dictionary.
1. `params` are the user-specified parameters used to make the request.
1. `response` is the object from the requests library, `requests.get(...)`.
1. `service` is the requested Synoptic API service type.
1. `token_source` is where SynopticPy found the token.
1. `url` is the full URL used to make the API request.
1. `verbose` indicates if details about what SynopticPy is doing is printed to the screen (i.e., poor-man's logging).

Let's take a look at the attributes of Metadata service instance...

In [20]:
# Get station metadata for a single, specific station
s = synoptic.Metadata(stid="KSLC", verbose=True)

print(f"{s.endpoint=}")
print(f"{s.service=}")
print(f"{s.help_url=}")
print(f"{s.token_source=}")
# print(f"{s.params=}")
# print(f"{s.url=}")
print(f"{s.verbose=}")
s.df

🚚💨 Speedy delivery from Synoptic metadata service.
📦 Received data from 1 stations.
s.endpoint='https://api.synopticdata.com/v2/stations/metadata'
s.service='metadata'
s.help_url='https://docs.synopticdata.com/services/weather-data-api'
s.token_source='Config File: /home/blaylock/.config/SynopticPy'
s.verbose=True


id,stid,name,elevation,latitude,longitude,status,mnet_id,state,timezone,elev_dem,period_of_record_start,period_of_record_end,restricted
u32,str,str,f64,f64,f64,str,u32,str,str,f64,"datetime[μs, UTC]","datetime[μs, UTC]",bool
53,"""KSLC""","""Salt Lake City, Salt Lake City…",4226.0,40.77069,-111.96503,"""ACTIVE""",1,"""UT""","""America/Denver""",4235.6,1997-01-01 00:00:00 UTC,2024-09-08 03:54:00 UTC,false


## What is the DataFrame structure?

SynopticPy returns all data as long-format Polars DataFrames. This means that for data requests, each row in the dataframe is a single unique observation.

Why? This makes it easy to archive the data locally, such as a Parquet file.

I will let the user manipulate the DataFrame in the way they want it using Polars' extensive and efficient processing. For instance, long-format DataFrames can be _pivoted_ to make a DataFrame with each column as a different variable.

In [36]:
df = synoptic.TimeSeries(
    stid="ukbkb",
    recent=timedelta(hours=6),
    vars=["air_temp", "wind_speed", "wind_direction"],
).df

df.head()

🚚💨 Speedy delivery from Synoptic timeseries service.
📦 Received data from 1 stations.


date_time,variable,sensor,derived,value,units,id,stid,name,elevation,latitude,longitude,status,mnet_id,state,timezone,elev_dem,period_of_record_start,period_of_record_end,restricted
"datetime[μs, UTC]",str,u32,bool,f64,str,u32,str,str,f64,f64,f64,str,u32,str,str,str,"datetime[μs, UTC]","datetime[μs, UTC]",bool
2024-09-07 23:15:00 UTC,"""air_temp""",1,false,35.0,"""Celsius""",37032,"""UKBKB""","""EW2355 Spanish Fork""",4734.0,40.09867,-111.62767,"""ACTIVE""",65,"""UT""","""America/Denver""","""4740.8""",2013-03-13 00:00:00 UTC,2024-09-08 04:45:00 UTC,false
2024-09-07 23:30:00 UTC,"""air_temp""",1,false,35.0,"""Celsius""",37032,"""UKBKB""","""EW2355 Spanish Fork""",4734.0,40.09867,-111.62767,"""ACTIVE""",65,"""UT""","""America/Denver""","""4740.8""",2013-03-13 00:00:00 UTC,2024-09-08 04:45:00 UTC,false
2024-09-07 23:45:00 UTC,"""air_temp""",1,false,35.0,"""Celsius""",37032,"""UKBKB""","""EW2355 Spanish Fork""",4734.0,40.09867,-111.62767,"""ACTIVE""",65,"""UT""","""America/Denver""","""4740.8""",2013-03-13 00:00:00 UTC,2024-09-08 04:45:00 UTC,false
2024-09-08 00:00:00 UTC,"""air_temp""",1,false,34.444,"""Celsius""",37032,"""UKBKB""","""EW2355 Spanish Fork""",4734.0,40.09867,-111.62767,"""ACTIVE""",65,"""UT""","""America/Denver""","""4740.8""",2013-03-13 00:00:00 UTC,2024-09-08 04:45:00 UTC,false
2024-09-08 00:15:00 UTC,"""air_temp""",1,false,33.333,"""Celsius""",37032,"""UKBKB""","""EW2355 Spanish Fork""",4734.0,40.09867,-111.62767,"""ACTIVE""",65,"""UT""","""America/Denver""","""4740.8""",2013-03-13 00:00:00 UTC,2024-09-08 04:45:00 UTC,false


In [38]:
df.pivot(index=["date_time", "stid"], on="variable", values="value")

date_time,stid,air_temp,wind_speed,wind_direction
"datetime[μs, UTC]",str,f64,f64,f64
2024-09-07 23:15:00 UTC,"""UKBKB""",35.0,0.0,null
2024-09-07 23:30:00 UTC,"""UKBKB""",35.0,0.448,40.0
2024-09-07 23:45:00 UTC,"""UKBKB""",35.0,0.0,null
2024-09-08 00:00:00 UTC,"""UKBKB""",34.444,0.0,null
2024-09-08 00:15:00 UTC,"""UKBKB""",33.333,0.0,null
…,…,…,…,…
2024-09-08 04:00:00 UTC,"""UKBKB""",23.333,0.448,328.0
2024-09-08 04:15:00 UTC,"""UKBKB""",23.333,0.895,45.0
2024-09-08 04:30:00 UTC,"""UKBKB""",22.778,0.0,null


## How can I split a long-format DataFrame by station?
Polars makes this easy; use `df.partition_by('stid')` to get a list of DataFrames, each DataFrame with it's own station.

In [43]:
df = synoptic.TimeSeries(
    stid="ukbkb,kslc",
    recent=timedelta(minutes=30),
    vars="air_temp",
).df

df.partition_by("stid")


🚚💨 Speedy delivery from Synoptic timeseries service.
📦 Received data from 2 stations.


[shape: (6, 20)
 ┌─────────────┬──────────┬────────┬─────────┬───┬──────────┬─────────────┬────────────┬────────────┐
 │ date_time   ┆ variable ┆ sensor ┆ derived ┆ … ┆ elev_dem ┆ period_of_r ┆ period_of_ ┆ restricted │
 │ ---         ┆ ---      ┆ ---    ┆ ---     ┆   ┆ ---      ┆ ecord_start ┆ record_end ┆ ---        │
 │ datetime[μs ┆ str      ┆ u32    ┆ bool    ┆   ┆ str      ┆ ---         ┆ ---        ┆ bool       │
 │ , UTC]      ┆          ┆        ┆         ┆   ┆          ┆ datetime[μs ┆ datetime[μ ┆            │
 │             ┆          ┆        ┆         ┆   ┆          ┆ , UTC]      ┆ s, UTC]    ┆            │
 ╞═════════════╪══════════╪════════╪═════════╪═══╪══════════╪═════════════╪════════════╪════════════╡
 │ 2024-09-08  ┆ air_temp ┆ 1      ┆ false   ┆ … ┆ 4235.6   ┆ 1997-01-01  ┆ 2024-09-08 ┆ false      │
 │ 04:50:00    ┆          ┆        ┆         ┆   ┆          ┆ 00:00:00    ┆ 04:10:00   ┆            │
 │ UTC         ┆          ┆        ┆         ┆   ┆          ┆ UTC 

## How to get Metadata for stations of interest?

In [44]:
synoptic.Metadata(radius="kmry,5").df

🚚💨 Speedy delivery from Synoptic metadata service.
📦 Received data from 34 stations.


id,stid,name,elevation,latitude,longitude,status,mnet_id,state,timezone,elev_dem,distance,period_of_record_start,period_of_record_end,restricted
u32,str,str,f64,f64,f64,str,u32,str,str,f64,f64,"datetime[μs, UTC]","datetime[μs, UTC]",bool
276,"""KMRY""","""Monterey Regional Airport""",167.0,36.59047,-121.84875,"""ACTIVE""",1,"""CA""","""America/Los_Angeles""",170.6,0.0,1997-04-12 00:00:00 UTC,2024-09-08 04:54:00 UTC,false
2478,"""DMB""","""BAMI1""",26.0,36.61,-121.87,"""INACTIVE""",31,"""CA""","""America/Los_Angeles""",0.0,1.79,2000-05-07 00:00:00 UTC,2003-02-06 21:50:00 UTC,false
2489,"""MBA""","""BAMI12""",75.0,36.62,-121.9,"""INACTIVE""",31,"""CA""","""America/Los_Angeles""",0.0,3.5,2000-05-07 00:00:00 UTC,2003-02-06 20:50:00 UTC,false
3619,"""RTGC1""","""FORT ORD #2""",490.0,36.626944,-121.786389,"""INACTIVE""",2,"""CA""","""America/Los_Angeles""",469.2,4.28,2001-10-11 00:00:00 UTC,2012-10-31 03:34:00 UTC,false
18515,"""CMEC1""","""CARMEL RIVER NEAR CARMEL 3E""",45.0,36.53917,-121.87944,"""INACTIVE""",106,"""CA""","""America/Los_Angeles""",52.5,3.93,2006-12-16 00:00:00 UTC,2007-12-28 18:15:00 UTC,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
155497,"""026PG""","""Parker Flats Cutoff""",436.0,36.62987,-121.79182,"""ACTIVE""",229,"""CA""","""America/Los_Angeles""",419.9,4.17,2020-08-25 03:14:00 UTC,2024-09-08 04:50:00 UTC,false
166603,"""G0246""","""GW0246 Monterey""",314.0,36.57217,-121.7975,"""ACTIVE""",65,"""CA""","""America/Los_Angeles""",301.8,3.11,2021-07-26 22:09:00 UTC,2024-09-08 04:54:00 UTC,false
236415,"""NDBC46240""","""CABRILLO POINT, MONTEREY BAY, …",0.0,36.626,-121.907,"""ACTIVE""",286,"""CA""","""America/Los_Angeles""",null,4.06,2024-04-25 11:26:00 UTC,2024-09-08 03:56:00 UTC,false


## What if I don't know Polars and love Pandas?
You can ...

In [48]:
synoptic.Metadata(radius="kmry,5").df.to_pandas()


🚚💨 Speedy delivery from Synoptic metadata service.
📦 Received data from 34 stations.


,id,stid,name,elevation,latitude,longitude,status,mnet_id,state,timezone,elev_dem,distance,period_of_record_start,period_of_record_end,restricted
0,276,KMRY,Monterey Regional Airport,167.00,36.590470,-121.848750,ACTIVE,1,CA,America/Los_Angeles,170.6,0.00,1997-04-12 00:00:00+00:00,2024-09-08 04:54:00+00:00,False
1,2478,DMB,BAMI1,26.00,36.610000,-121.870000,INACTIVE,31,CA,America/Los_Angeles,0.0,1.79,2000-05-07 00:00:00+00:00,2003-02-06 21:50:00+00:00,False
2,2489,MBA,BAMI12,75.00,36.620000,-121.900000,INACTIVE,31,CA,America/Los_Angeles,0.0,3.50,2000-05-07 00:00:00+00:00,2003-02-06 20:50:00+00:00,False
3,3619,RTGC1,FORT ORD #2,490.00,36.626944,-121.786389,INACTIVE,2,CA,America/Los_Angeles,469.2,4.28,2001-10-11 00:00:00+00:00,2012-10-31 03:34:00+00:00,False
4,18515,CMEC1,CARMEL RIVER NEAR CARMEL 3E,45.00,36.539170,-121.879440,INACTIVE,106,CA,America/Los_Angeles,52.5,3.93,2006-12-16 00:00:00+00:00,2007-12-28 18:15:00+00:00,False
5,18962,C7571,CW7571 Pacific Grove,328.00,36.599000,-121.898170,INACTIVE,65,CA,America/Los_Angeles,62.3,2.80,2007-02-21 00:00:00+00:00,2007-05-25 01:39:00+00:00,False
6,23467,MTYC1,Monterey,0.00,36.605000,-121.888000,ACTIVE,122,CA,America/Los_Angeles,0.0,2.40,2008-03-20 00:00:00+00:00,2024-09-08 04:48:00+00:00,False
7,25037,46240,Cabrillo Point,0.00,36.626000,-121.907000,ACTIVE,96,CA,America/Los_Angeles,0.0,4.06,2008-12-10 00:00:00+00:00,2024-09-08 04:26:00+00:00,False
8,25348,CI210,Carmel,75.00,36.540000,-121.880000,ACTIVE,66,CA,America/Los_Angeles,45.9,3.89,2009-01-28 00:00:00+00:00,2024-09-08 02:00:00+00:00,False
9,31842,CI229,Laguna Seca,320.00,36.570000,-121.786389,ACTIVE,66,CA,America/Los_Angeles,305.1,3.74,2011-10-27 00:00:00+00:00,2024-09-08 02:00:00+00:00,False
